In [25]:
import tweepy
import csv, os
import threading, random

consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

tweetsPerQry = 100
maxTweets = 100
# Add 5 hashtag comma separated
hashtag = ""

authentication = tweepy.OAuthHandler(consumer_key, consumer_secret)
authentication.set_access_token(access_token, access_token_secret)
api = tweepy.API(authentication, wait_on_rate_limit=True)
maxId = -1
tweetCount = 0

tweetIds = set()

def writeTweets(filename, tag):
  global tweetCount, maxId, tweetIds
  fileExists = os.path.exists(filename)

  if fileExists:
    with open(filename, 'r', newline='') as file:
      reader = csv.reader(file)
      next(reader, None)
      for row in reader:
        tweetIds.add(str(row[0]))
  with open(filename, 'a', newline='', encoding='utf8') as csvFile:
    fieldNames = ['id', 'tweet', 'time', 'lat', 'long']
    writer = csv.DictWriter(csvFile, fieldnames=fieldNames)
    if not fileExists:
        writer.writeheader()
    while tweetCount < maxTweets:
      if(maxId <= 0):
        # newTweets = api.search(q=hashtag, count=tweetsPerQry, result_type="recent", tweet_mode="extended")
        newTweets = api.search_tweets(q=tag, count=tweetsPerQry, result_type="recent", tweet_mode="extended")
      else:
        newTweets = api.search_tweets(q=tag, count=tweetsPerQry, max_id=str(maxId - 1), result_type="recent", tweet_mode="extended")
      
      # if not newTweets:
      #   print("Tweet Habis")
      #   break
      
      for tweet in newTweets:
        # if str(tweet.id) not in tweetIds and (tweet.coordinates or tweet.user.location):
        if str(tweet.id) not in tweetIds:
          lat = random.uniform(24.396308, 49.384358)
          long = random.uniform(-124.848974, -66.885444)
          tweetIds.add(tweet.id)
          writer.writerow({'id': tweet.id, 'tweet': tweet.full_text.encode(encoding='utf8'), 'time': tweet.created_at, 'lat': lat, 'long': long})
          # print("id:", tweet.id, "tweet:", tweet.full_text.encode(encoding='utf8'), "time:", tweet.created_at, "geo:", tweet.user.location, "enabled:", tweet.user.geo_enabled)
        
      tweetCount += len(newTweets)
      maxId = newTweets[-1].id
print("Length of tweets: ", len(tweetIds))

threads = []
tags = hashtag.split(',')
for i in range(8):
  filename = "twitterSportsData" + str(i) + '.csv'
  t = threading.Thread(target=writeTweets, args = (filename,tags[i],))
  threads.append(t)
  t.start()

for t in threads:
  t.join()

Length of tweets:  0
